In [1]:
!pip install tflearn 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=873561fb9bddaec61211fa04123a6734d6b3c742e54e6caa859e2df026d65f8b
  Stored in directory: /root/.cache/pip/wheels/55/fb/7b/e06204a0ceefa45443930b9a250cb5ebe31def0e4e8245a465
Successfully built tflearn


In [2]:
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 79.3 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=130d76176b0be5820659bf8a652e9767983f3281f5fd0fbed25e7117489b7997
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [3]:
!pip install pyspellchecker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 36.7 MB/s eta 0:00:00


In [4]:
import numpy as np
import tensorflow as tf
import tflearn
import random

import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import json
import pickle
import warnings
warnings.filterwarnings("ignore")

Instructions for updating:
non-resource variables are not supported in the long term


In [5]:
#check to see if json is loading correctly
import json
import chardet

with open('physics.json', 'rb') as file:
    raw_data = file.read()
    detected_encoding = chardet.detect(raw_data)['encoding']

with open('physics.json', encoding=detected_encoding) as file:
    physics = json.load(file)

In [6]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [8]:
from spellchecker import SpellChecker
spell = SpellChecker()

In [9]:
import json
import pickle
import random
import numpy as np
import nltk
from nltk.stem.lancaster import LancasterStemmer
import tflearn
import tensorflow as tf

stemmer = LancasterStemmer()
import json
import chardet

with open('physics.json', 'rb') as file:
    raw_data = file.read()
    detected_encoding = chardet.detect(raw_data)['encoding']

with open('physics.json', encoding=detected_encoding) as file:
    physics = json.load(file)


# Initialize variables
words = []
classes = []
documents = []
ignore_words = ['?']

# Process physics and create words, classes, documents, and ignore_words
for intent in physics['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# Stemming, Lowering and Removing Duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

# Create training data
training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]

    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

# Shuffle and convert training data to numpy array
random.shuffle(training)
training = np.array(training)

# Split training data into input and output
train_x = list(training[:, 0])
train_y = list(training[:, 1])

# Define neural network model
tf.compat.v1.reset_default_graph()
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Train and save the model
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

# Save training data to a pickle file
pickle.dump({'words': words, 'classes': classes, 'train_x': train_x, 'train_y': train_y}, open("training_data", "wb"))

# Load pickle data
data = pickle.load(open("training_data", "rb"))
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

# Load the saved model
model.load('./model.tflearn')

# Define helper functions
def correct_sentence_spelling(sentence_words):
    corrected_words = [spell.correction(word) for word in sentence_words]
    return corrected_words

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = correct_sentence_spelling(sentence_words)
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)
    return np.array(bag)

ERROR_THRESHOLD = 0.25

def find_most_similar_question(query, threshold=0.8):
    query_embedding = sbert_model.encode(query)
    max_similarity = 0
    most_similar_question = None

    for intent in physics['intents']:
        for question in intent['patterns']:
            question_embedding = sbert_model.encode(question)
            similarity = np.inner(query_embedding, question_embedding)

            if similarity > max_similarity and similarity >= threshold:
                max_similarity = similarity
                most_similar_question = question

    return most_similar_question

def classify(sentence):
    most_similar_question = find_most_similar_question(sentence)
    if most_similar_question is not None:
        results = model.predict([bow(most_similar_question, words)])[0]
    else:
        results = model.predict([bow(sentence, words)])[0]

    results = [[i, r] for i, r in enumerate(results) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    if results:
        while results:
            for i in physics['intents']:
                if i['tag'] == results[0][0]:
                    return print(random.choice(i['responses']))
            results.pop(0)

Training Step: 65999  | total loss: 0.01652 | time: 0.402s
| Adam | epoch: 1000 | loss: 0.01652 - acc: 0.9933 -- iter: 520/528
Training Step: 66000  | total loss: 0.01487 | time: 0.406s
| Adam | epoch: 1000 | loss: 0.01487 - acc: 0.9815 -- iter: 528/528
--


In [9]:
while True:
    input_data = input("input- ")
    answer = response(input_data)
    answer




input- define physics?
Physics is the study of matter, energy, and their interactions. It covers a wide range of topics including mechanics, thermodynamics, electromagnetism, and quantum mechanics.
input- hi
Hello, thanks for visiting
input- defene physics?
Physics is the study of matter, energy, and their interactions. It covers a wide range of topics including mechanics, thermodynamics, electromagnetism, and quantum mechanics.


KeyboardInterrupt: ignored